In [4]:
from bs4 import BeautifulSoup
from dateutil.parser import parse
from datetime import datetime
from datetime import datetime, timedelta
import urlparse
import pymongo 
from pymongo import MongoClient
import os

In [5]:
# def get_draft_date(soup):
#     draft_deets = soup.find("div", {"class": "games-alert-mod alert-mod2 games-grey-alert"})
#     draft_date = None
#     for b in draft_deets.findAll('b'):
#         key = b.text.replace(':', '').strip()
#         val = b.nextSibling.strip()
#         if key=='Draft Date':
#             val = b.nextSibling.replace(',','').replace('.','')[4:].strip()
#             draft_date = datetime.strptime(val, '%b %d %Y')
#     return draft_date

In [6]:
def get_league_draft_picks(soup):
    
#     draft_date = get_draft_date(soup)
    draft = soup.find("div", {"class": "games-fullcol games-fullcol-extramargin"})
    
    draft_deets = draft.find("div", {"class": "games-alert-mod alert-mod2 games-grey-alert"})
    for deet in draft_deets.findAll('b'):
        if str(deet.text).strip() == "Type:":
            draft_type=deet.nextSibling.strip()
            #print draft_type
            if draft_type != "Snake Draft":
                return []
    
    draft_results=draft.find('table')
    #print draft_results
    league_picks = []
    
    for round_num, draft_round in enumerate(draft_results.findAll('table')):
        
        for pick in draft_round.findAll('tr',{"class":"tableBody"}):
            pick_dict = {}
            for i, td in enumerate(pick.findAll('td')):
                if i == 0:
                    pick_num= int(td.text)
                elif i == 1: 
                    player_link = td.find('a')
                    if not player_link == None:
                        player_name = player_link.text
                        player_id = player_link['playerid']
                        unique_team_id = player_link['teamid']
                    else:
                        player_name = ''
                        player_id = -9999999999
                        unique_team_id = -9999999999
                elif i == 2:
                    team = td.find('a')
                    team_link = team['href']
                    team_name=team.text
                    parsed=urlparse.urlparse(team_link)
                    team_id=urlparse.parse_qs(parsed.query)['teamId'][0]
                    season_id = urlparse.parse_qs(parsed.query)['seasonId'][0]
                    league_id = urlparse.parse_qs(parsed.query)['leagueId'][0]
                    
                    #team_id = team_link['teamId']
                    #print team_id
                else:
                    raise

            pick_dict = {"league_id":league_id,
                         #"draft_date":draft_date,
                        "team_name":team_name,
                        "team_id":team_id,
                        "unique_team_id":unique_team_id,
                        "season_id":season_id,
                        "player_name":player_name,
                        "player_id":player_id,
                        "pick_num":int(pick_num),
                        "round_num":round_num+1}

            league_picks.append(pick_dict)
    
    return league_picks

In [7]:
def get_all_picks(year):

    all_picks = []

    for draft_recap in os.listdir(year):
        if not draft_recap.startswith("."):
            #print draft_recap
            draft_file = open('%s/%s'%(year,draft_recap),'r')
            soup = BeautifulSoup(draft_file,"lxml")
            all_picks = all_picks + get_league_draft_picks(soup)
            draft_file.close()
            
    return all_picks

In [8]:
#get_all_picks('2015')

In [10]:
connection = MongoClient()
db = connection['espn_leagues_drafts']
drafts = db['snake_drafts']
drafts.drop()
connection = MongoClient()
db = connection['espn_leagues_drafts']
drafts = db['snake_drafts']
drafts.insert_many(get_all_picks('2015'))

OSError: [Errno 2] No such file or directory: '2015'